In [ ]:
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
players_salary = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_salary.csv')
players_stats = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_players_stats_combined.csv')
players_edsm = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_endorsements.csv')
players_twitter = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_twitter_players.csv')
players_RPM = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_real_plus_minus.csv')
team_value = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_att_val.csv')

In [ ]:
team_value

In [ ]:
players_salary

In [ ]:
players_stats = players_stats.rename(columns={'PLAYER':'NAME'})
players_stats.head(8)

In [ ]:
players_RPM = players_stats[['NAME','GP','MPG','ORPM','DRPM','RPM','WINS_RPM']]
players_RPM

In [ ]:
players_sal_RPM = pd.merge(players_salary,players_RPM,how='inner',on='NAME')
players_sal_RPM

In [ ]:
players_sal_RPM_TV = pd.merge(players_sal_RPM,team_value[['TEAM','VALUE_MILLIONS']],how='inner',on='TEAM')
players_sal_RPM_TV

* It's natural for us to have an idea that the salary a players can get mainly depends on his performance in a match.
* The performance of a player can be described as RPM.
* And I wanna know the correlation between SALARY and RPM.

In [ ]:
corr1 = players_sal_RPM_TV.corr()
corr1

In [ ]:
sns.heatmap(corr1)

In [ ]:
results = smf.ols('SALARY ~MPG',data=players_sal_RPM_TV).fit()
results.summary()

In [ ]:
results = smf.ols('SALARY ~WINS_RPM',data=players_sal_RPM_TV).fit()
results.summary()

In [ ]:
results = smf.ols('SALARY ~RPM',data=players_sal_RPM_TV).fit()
results.summary()

* it seems that the salary a player can get has closer relation to his MPG than RPM.
* Team value almost has nothing to do with the salary a player can get.
* Actually we know that if a player can make more contribution to win,he will get more time on the court.
* I wanna know if the players with high salary bring positive influence on the court all the time.
* Set up new feature: "RPM per MPG","WIN_RPM per MPG"

In [ ]:
players_sal_RPM.eval('WRPMpM = WINS_RPM / MPG',inplace = True)
players_sal_RPM.eval('RPMpM = RPM / MPG',inplace = True)
players_sal_RPM

In [ ]:
corr = players_sal_RPM.corr()
corr

In [ ]:
sns.heatmap(corr)

It's interesting that WRPMpM and RPMpM has less strong correlation than MPG and WIN_RPM.
So the contribution and performance a player have made in his past career have a more important influence on his salary.

Next I wanna see the influence made by social media and their fans,
and the correlation between endorsement and social influence.

In [ ]:
players_twitter = players_twitter.rename(columns={'PLAYER':'NAME'})

In [ ]:
players_sal_twi = pd.merge(players_salary,players_twitter,how='inner',on='NAME')
players_sal_twi

In [ ]:
corr2 = players_sal_twi.corr()
corr2

In [ ]:
sns.heatmap(corr2)

We can know that the salary will not be raised because you have more fans than others.Managers focus on your performance in a game.

In [ ]:
players_edsm

In [ ]:
players_edsm['Endorsement'] = players_edsm['ENDORSEMENT'].str[1:]
players_sal_edsm_twi = pd.merge(players_sal_twi,players_edsm[['NAME','Endorsement']],how='inner',on='NAME')
players_sal_edsm_twi

In [ ]:
def convert(item):
    s = ''
    temp = item.strip().split(',')
    for i in range(len(temp)):
        s += temp[i]
    return float(s)
players_sal_edsm_twi['Endorsement'] = players_sal_edsm_twi['Endorsement'].map(convert)

In [ ]:
players_sal_edsm_twi.info()

In [ ]:
players_sal_edsm_twi

In [ ]:
corr2 = players_sal_edsm_twi.corr()
corr2

In [ ]:
sns.heatmap(corr2)

In [ ]:
result = smf.ols('Endorsement ~TWITTER_FAVORITE_COUNT',data=players_sal_edsm_twi).fit()
result.summary()

The correlation between Endorsement and TWITTER_FAVORITE_COUNT is close to our assumption.

I still wanna know
* The correlation between endorsement and performance.
* What's the main reason of a man being a fan of a player.

In [ ]:
players_data = pd.merge(players_sal_edsm_twi,players_sal_RPM[['NAME','MPG','ORPM','WINS_RPM','WRPMpM']],how='inner',on='NAME')
players_data.eval('ORPMpM = ORPM / MPG',inplace = True)
players_data

In [ ]:
corr = players_data.corr()
corr

In [ ]:
sns.heatmap(corr)

We can see that WINS_RPM has the strongest correlation to Endorsement.That means the enterprises pay more attention to the contribution and positive influence the player bring to every match.These performances and visual impact are obvious and direct to the audience and their customers.

We can also see that WRPMpM is much more important for them to appreciate a player than MPG.And WRPMpM has stronger correlation to TWITTER_FAVORITE_COUNT than WINS_RPM,so does ORPMpM.Therefore,as for NBA fans,they appreciate the player who exert all effort to win and perform well all the time when he is on the court.